In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# import lightgbm
from lightgbm import LGBMRegressor
import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [44]:
train_path = './trainingdata'
infer_path = './inferencedata'
submission_path = './submission'

In [60]:
submission = pd.read_csv('./data/sample_submission.csv')

In [61]:
time_str = '20201226_003554'
train = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)
X_test = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}.csv')
)

# Metric

In [62]:
def pinball(pred, actual):
    quantile_col = pred.columns.tolist()
    pred2 = pred.copy()
    actual2 = actual.copy()
    
    pred2.index = range(len(pred2))
    actual2.index = range(len(actual2))

    pinball_loss = []
    for item in quantile_col:
        pre_pinball = []
        for idx in range(len(pred2)):
            y = actual2.iloc[idx]
            z = pred2[item].iloc[idx]
            q = item
            
            if y >= z:
                value = (y - z)*q
                pre_pinball.append(value)
            else:
                value = (z - y)*(1 - q)
                pre_pinball.append(value)
        loss_mean = np.mean(pre_pinball)
        pinball_loss.append(loss_mean)
    
    fin = np.mean(pinball_loss)
#     print(f'Pinball Loss: {fin}')
    
    return pinball_loss         

# Inference

In [63]:
X_train = train.drop(columns = ['Target1', 'Target2'])
target_1 = train['Target1']
target_2 = train['Target2']

In [64]:
def LGBM_reg(X_train,y_train, test):
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 랜덤모수 샘플 추론이 왜 안되지;;;
    # fold = StratifiedKFold(n_splits=5, shuffle = True)
    
    fold = KFold(n_splits=5, shuffle = True)

    oof_preds = np.zeros([X_train.shape[0], len(quantiles)])
    
    feature_importance_df = pd.DataFrame()
    
    test_df = pd.DataFrame(np.zeros([test.shape[0], len(quantiles)]))
    test_df.columns = quantiles
    
    fold_metric = []
    # training
    target = y_train
    for fold_, (train_idx, valid_idx) in enumerate(
        fold.split(np.array(X_train), target)
    ):

        train_x, train_y = X_train.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = X_train.iloc[valid_idx], target.iloc[valid_idx]

        oof_df = pd.DataFrame()
        for idx, q in enumerate(quantiles):
            print(f'\nquantile: {q}\n')
            model = LGBMRegressor(objective='quantile', alpha=q,
                                 n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)                   

            model.fit(train_x, train_y, eval_metric = ['quantile'], 
              eval_set=[(valid_x, valid_y)], early_stopping_rounds=300, verbose=1500)

            oof_preds[[valid_idx],idx] = \
                model.predict(valid_x).round(2)

            oof_df[q] = model.predict(valid_x).round(2)
            test_df[q] += model.predict(test).round(2) / fold.n_splits
            
        metric = pinball(oof_df, valid_y)
        loss = np.mean(metric)
        print(f'\n FOLD {fold_}의 total pinball loss: {loss}\n')
        fold_metric.append(metric)
        # feature importance
        
        print('\n===================================================')
        print(f'FOLD {fold_} Success')
        print('===================================================\n')
        
    return oof_preds, fold_metric, test_df

In [65]:
oof_preds, fold_metric, test_df = \
    LGBM_reg(X_train, target_1, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[772]	valid_0's quantile: 1.33684

quantile: 0.2

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[1113]	valid_0's quantile: 2.1076

quantile: 0.3

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[1125]	valid_0's quantile: 2.50787

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.64662
Early stopping, best iteration is:
[1801]	valid_0's quantile: 2.63859

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.60291
[3000]	valid_0's quantile: 2.58078
[4500]	valid_0's quantile: 2.55592
[6000]	valid_0's quantile: 2.54891
[7500]	valid_0's quantile: 2.54403
[9000]	valid_0's quantile: 2.53902
Did not meet early stopping. Best iteration is:
[9881]	valid_0's quantile: 2.53808

qua

[1500]	valid_0's quantile: 0.772052
[3000]	valid_0's quantile: 0.770014
Early stopping, best iteration is:
[2821]	valid_0's quantile: 0.769655

 FOLD 3의 total pinball loss: 1.9312755719338588


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[768]	valid_0's quantile: 1.34184

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.09764
[3000]	valid_0's quantile: 2.09326
Early stopping, best iteration is:
[3617]	valid_0's quantile: 2.09228

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.48418
[3000]	valid_0's quantile: 2.47132
[4500]	valid_0's quantile: 2.45971
[6000]	valid_0's quantile: 2.45154
Early stopping, best iteration is:
[6747]	valid_0's quantile: 2.4484

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.58989
[3000]	valid_0's quantile

In [66]:
oof_preds2, fold_metric2, test_df2 = \
    LGBM_reg(X_train, target_2, X_test)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.37497
Early stopping, best iteration is:
[1591]	valid_0's quantile: 1.37491

quantile: 0.2

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[581]	valid_0's quantile: 2.22279

quantile: 0.3

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[908]	valid_0's quantile: 2.63332

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.78863
[3000]	valid_0's quantile: 2.72492
[4500]	valid_0's quantile: 2.71114
Early stopping, best iteration is:
[5631]	valid_0's quantile: 2.70471

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.66688
[3000]	valid_0's quantile: 2.63607
[4500]	valid_0's quantile: 2.62463
[6000]	valid_0's quantile: 2.61469
Early stopping, best iteration is:
[6884]	valid_0's 

Early stopping, best iteration is:
[277]	valid_0's quantile: 1.41206

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.24004
[3000]	valid_0's quantile: 2.2182
Early stopping, best iteration is:
[3269]	valid_0's quantile: 2.21566

quantile: 0.3

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[750]	valid_0's quantile: 2.63966

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.72466
Early stopping, best iteration is:
[1667]	valid_0's quantile: 2.72297

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.65157
[3000]	valid_0's quantile: 2.62029
[4500]	valid_0's quantile: 2.61186
[6000]	valid_0's quantile: 2.60552
[7500]	valid_0's quantile: 2.59987
Early stopping, best iteration is:
[7596]	valid_0's quantile: 2.59966

quantile: 0.6

Training until validation scores don't improv

# submission

In [67]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = test_df.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = test_df2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
time_str = datetime.datetime.strftime(
    datetime.datetime.today(),
    '%Y%m%d_%H%M%S'
)

submission.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}.csv'), 
    index=False
)